In [12]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:

    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.6.1

In [13]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [14]:
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import ast
import re
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq,AutoTokenizer
import torch
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [15]:
from google.colab import files
data=files.upload()
data=files.upload()
data=files.upload()

Saving test.csv to test.csv


Saving validation.csv to validation.csv


Saving train.csv to train.csv


In [16]:

train_data=pd.read_csv('train.csv')
validation_data=pd.read_csv('validation.csv')
test_data=pd.read_csv('test.csv')

In [17]:
train_data.head()

,dialog,act,emotion
0,"['Say , Jim , how about going for a few beers ...",[3 4 2 2 2 3 4 1 3 4],[0 0 0 0 0 0 4 4 4 4]
1,"['Can you do push-ups ? '\n "" Of course I can ...",[2 1 2 2 1 1],[0 0 6 0 0 0]
2,"['Can you study with the radio on ? '\n ' No ,...",[2 1 2 1 1],[0 0 0 0 0]
3,['Are you all right ? '\n ' I will be all righ...,[2 1 1 1],[0 0 0 0]
4,"['Hey John , nice skates . Are they new ? '\n ...",[2 1 2 1 1 2 1 3 4],[0 0 0 0 0 6 0 6 0]


In [18]:
validation_data.head()

,dialog,act,emotion
0,"['Good morning , sir . Is there a bank near he...",[2 1 3 2 1 2 1],[0 0 0 0 0 0 0]
1,['Good afternoon . This is Michelle Li speakin...,[2 1 1 1 1 2 3 2 3 4],[0 0 0 0 0 0 0 0 0 0]
2,['What qualifications should a reporter have ?...,[2 1 2 1],[0 0 0 0]
3,"['Hi , good morning , Miss ? what can I help y...",[2 3 2 2 1 2 1 2 1 1 1 3 2 1 3 4],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
4,"[""Excuse me , ma'am . Can you tell me where th...",[3 4 2 1 2 1 1 1],[0 0 0 0 0 0 4 4]


In [19]:
test_data.head()

,dialog,act,emotion
0,"['Hey man , you wanna buy some weed ? ' ' Some...",[3 2 3 4 3 4 3 2 3 4 2 3],[0 6 0 0 0 0 0 0 0 0 3 0]
1,['The taxi drivers are on strike again . ' ' W...,[1 2 1 1],[0 0 0 0]
2,"[""We've managed to reduce our energy consumpti...",[1 2 1 2 1 2 1],[0 0 0 0 0 0 0]
3,"['Believe it or not , tea is the most popular ...",[1 1 1 1 2 2 2 2 1 1 1 3 4 3],[0 0 0 0 0 0 0 0 0 4 0 0 4 4]
4,['What are your personal weaknesses ? '\n ' I ...,[2 1 2 1 2 1 2 1],[0 0 0 0 0 0 0 4]


In [20]:
model_name ="unsloth/Mistral-Small-Instruct-2409"

In [21]:
def prepare_dataframe_for_generative_llm_finetuning(
    input_df,
    llm_tokenizer_name
):
    # Helper to fix string representations of lists
    def _fix_string_list_format(s):
        if isinstance(s, str):
            # seperating the strings
            s = re.sub(r"'\s+'", "', '", s)

            #add commas to list where needed and if list looks like number it changes it
            if re.match(r'^\[\s*\d+\s*(?:\d+\s*)*\]$', s.strip()):
                content = s.strip()[1:-1] # Remove outer brackets
                # Split by space, filter empty strings, join with comma
                s = '[' + ', '.join(filter(None, content.split(' '))) + ']'

            # Ensure it's enclosed in brackets, often a common issue for ast.literal_eval
            if not s.startswith('['): s = '[' + s
            if not s.endswith(']'): s = s + ']'
        return s

    # Hcleans the text
    def _clean_utterance_text(text):
        if not isinstance(text, str): text = str(text)
        cleaned_text = re.sub(r'<[^>]+>', '', text).strip()
        cleaned_text = re.sub(r'\[.*?\]', '', cleaned_text).strip()
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        return cleaned_text if cleaned_text else "Empty utterance."

    # emotion names from the database
    emotion_map = {
        0: 'no emotion',
        1: 'anger',
        2: 'disgust',
        3: 'fear',
        4: 'happiness',
        5: 'sadness',
        6: 'surprise'
    }
    intent_map = {
        0: '__dummy__', # Official label from the schema
        1: 'inform',
        2: 'question',
        3: 'directive',
        4: 'commissive'
    }

    tokenizer = AutoTokenizer.from_pretrained(llm_tokenizer_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dataset = Dataset.from_pandas(input_df)

    dataset = dataset.map(lambda examples: {
        'dialog': [_fix_string_list_format(s) for s in examples['dialog']],
        'emotion': [_fix_string_list_format(s) for s in examples['emotion']],
        'act': [_fix_string_list_format(s) for s in examples['act']]
    }, batched=True)

    def _create_conversation_chains_with_labels(examples):
        all_conversations = []
        for dialog_str, emotion_str, act_str in zip(examples["dialog"], examples["emotion"], examples["act"]):
            try:
                dialog_turns = ast.literal_eval(dialog_str)
                emotion_labels = ast.literal_eval(emotion_str)
                act_labels = ast.literal_eval(act_str)

                if not (dialog_turns and len(dialog_turns) == len(emotion_labels) == len(act_labels)):
                    all_conversations.append([])
                    continue

                current_chain = []
                for i, turn_text_raw in enumerate(dialog_turns):
                    cleaned_text = _clean_utterance_text(turn_text_raw)
                    role = "user" if i % 2 == 0 else "assistant"

                    if cleaned_text == "Empty utterance.":
                        continue

                    if role == "user":
                        current_emotion = emotion_map.get(emotion_labels[i], 'unknown_emotion')
                        current_intent = intent_map.get(act_labels[i], 'unknown_intent')
                        content_with_labels = (
                            f"Detected Mood: {current_emotion}.\n"
                            f"Detected Intent: {current_intent}.\n"
                            f"User: {cleaned_text}"
                        )
                        current_chain.append({"role": role, "content": content_with_labels})
                    else:
                        current_chain.append({"role": role, "content": cleaned_text})

                all_conversations.append(current_chain)
            except (ValueError, SyntaxError, TypeError):
                all_conversations.append([])
        return {"conversations": all_conversations}

    dataset = dataset.map(
        _create_conversation_chains_with_labels,
        batched=True,
        remove_columns=[col for col in input_df.columns if col != 'conversations'],
    )
    dataset = dataset.filter(lambda example: len(example['conversations']) > 0)

    def _apply_chat_template(examples):
        convos = examples["conversations"]
        texts = [tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False) for c in convos]
        return {"text": texts}

    dataset = dataset.map(
        _apply_chat_template,
        batched=True,
        remove_columns=['conversations'],
    )
    return dataset


In [22]:
dataset=prepare_dataframe_for_generative_llm_finetuning(
    input_df=train_data,
    llm_tokenizer_name = model_name
)

validation_dataset=prepare_dataframe_for_generative_llm_finetuning(
    input_df=validation_data,
    llm_tokenizer_name = model_name
)

test_dataset=validation_dataset=prepare_dataframe_for_generative_llm_finetuning(
    input_df=test_data,
    llm_tokenizer_name = model_name
)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [23]:

test_dataset

Dataset({
    features: ['text'],
    num_rows: 300
})

In [24]:

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Small-Instruct-2409",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "your token here",
)

==((====))==  Unsloth 2025.6.1: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/232k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [25]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.1 patched 56 layers with 56 QKV layers, 56 O layers and 56 MLP layers.


In [26]:
model.print_trainable_parameters()

trainable params: 95,420,416 || all params: 22,342,703,104 || trainable%: 0.4271


In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model),
    dataset_num_proc = 1,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 16, #gradient calculated after 16 steps
        warmup_steps = 5,
        num_train_epochs = 1
        max_steps = 150, #maximum steps
        learning_rate = 2e-4, #initial learning rate
        fp16 = True, #supported in t4 gpu
        bf16 = False, #not supported in t4 gpu
        logging_steps = 1,
        optim = "adamw_8bit", #adam optimiser
        weight_decay = 0.01, # L2 regularization
        lr_scheduler_type = "linear", # Learning rate scheduler type
        seed = 3407, # For reproducibility
        output_dir = "/content/drive/MyDrive/task3",
 # Directory to save checkpoints and final model
        report_to = "none",
        save_strategy="steps", # saving the checkpoints steps
        save_steps=20, # after every 20 steps
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/3548 [00:00<?, ? examples/s]

In [28]:
from unsloth.chat_templates import train_on_responses_only
# mistral format for chat template
trainer = train_on_responses_only(
    trainer,
    instruction_part = "[INST]", # Start of user instruction
    response_part = "[/INST]",   # End of user instruction, start of assistant response
)

Map (num_proc=2):   0%|          | 0/3548 [00:00<?, ? examples/s]

In [29]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<s><s>[INST] Detected Mood: no emotion.\nDetected Intent: inform.\nUser: Hi Bill , I saw your grandma yesterday .[/INST] Oh where was that ?</s>[INST] Detected Mood: no emotion.\nDetected Intent: inform.\nUser: I was running around the track at my college and there she was walking around the same track .[/INST] Grannie always tries to stay fit and healthy.She is always making us kids eat the proper foods .</s>[INST] Detected Mood: no emotion.\nDetected Intent: question.\nUser: Well , it pays off for her.How old is she anyway ?[/INST] She will be 86 next month .</s>[INST] Detected Mood: surprise.\nDetected Intent: inform.\nUser: That is truly amazing ![/INST]'

In [30]:

space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                  Oh where was that ?</s>                                         Grannie always tries to stay fit and healthy.She is always making us kids eat the proper foods .</s>                                    She will be 86 next month .</s>                         '

In [31]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
11.646 GB of memory reserved.


In [32]:
trainer_stats = trainer.train(resume_from_checkpoint=True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,548 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 95,420,416/11,420,735,488 (0.84% trained)
	save_steps: 10 (from args) != 20 (from trainer_state.json)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
41,1.893200
42,1.819400
43,1.781800
44,1.829800
45,1.653200
46,1.840100
47,1.745200
48,1.672000
49,1.639800
50,1.853000


In [33]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5539.862 seconds used for training.
92.33 minutes used for training.
Peak reserved memory = 12.971 GB.
Peak reserved memory for training = 1.325 GB.
Peak reserved memory % of max memory = 87.993 %.
Peak reserved memory for training % of max memory = 8.989 %.


In [34]:
# After trainer.train() completes
save_path = "/content/drive/MyDrive/task3/fine_tuned_model"
trainer.save_model(save_path)
print(f"Fine-tuned LoRA model saved to: {save_path}")

Fine-tuned LoRA model saved to: /content/drive/MyDrive/task3/fine_tuned_model
